In [ ]:
import json
import argparse
import re
from sympy import EX
from prompt import get_prompt, get_task_name, get_task_description
from response import get_batch_response
import csv
import os
import pandas as pd
from find_path import find_path
from utils import get_model_batch_response
import math
import tqdm 

prompt_t = """
    I am training a model using RoBERTa + MLP on a task named {task_name}. The task involves {task_description}. 
    Your task is to identify potential spurious patterns that the model might have learned based on its responses.

    I will present you with an instance where the model provided incorrect responses. 

    Please provide {spurious_num} assumptions of spurious patterns that may have caused the incorrect response. 
    Each assumption should be followed by {generate_num} verification data points to determine whether the model consistently makes mistakes 
    due to such a spurious pattern. The verification data should align with the identified spurious patterns. 
    Having the same pattern does not mean copying the original text and target verbatim; instead, 
    it should reflect the same pattern at a higher level of abstraction and  "text" and "target" in the generated should be diverse with various contents and different speaking way, and include spurious patterns.

    A spurious pattern refers to a misleading or non-causal feature relationship that the model learns during training, 
    such as misunderstandings of certain phrases, sentiment words, or entity relations.Be specific in the patterns, such as what words or what relations, but not a general description.

    Format your evaluation instances using XML tags. Each <Spurious_i> tag should include:

    An assumption of the spurious pattern that the model may have learned.
    {generate_num} verification data points, each enclosed in <verification_i> from <verification_1> to <verification_10>, where i is the sequential number of the verification set.
    Each <verification_i> should contain the following:

       1. <text>: A multi-sentence passage containing the spurious pattern. Generate the sample as long as the incorrect instance in length, at least 100 words in each data with a suitable context.
       2. <target>: An entity mentioned in the text.
       3. <ground_truth>: The true label for the classification task.
    Ensure that the ground truth of the generated data is ascertainably correct. If the correctness of the given instance cannot be determined, leave the field blank.
    The incorrect instance is as follows:
    {data}
    Please output all content completely without omitting or summarizing.
    Confirm that the generated data should be diverse to avoid overfitting of the smaller model.
    """

def format_prompt(data,prompt, task,description,spurious_num,generate_num):
    # NOTE
    # model's label { "FAVOR": 0  , "NONE": 1 , "AGAINST":  2}
    # the data is from model's inference
    # but that is needed to transform to { "FAVOR": 1, "NONE": 2, "AGAINST": 0}
    transform_dict = {
        0: 'FAVOR',
        1: 'NONE',
        2: 'AGAINST'
    }
    prompts = []
    for d in data:
        d_ = {
            "input_text": d["input_text"],
            "target": d["target"],
            "predicted_label": transform_dict[d["predicted_label"]],
            "true_label": transform_dict[d["true_label"]]
        }
        prompts.append(prompt.format(task_name = task, task_description = description, spurious_num = spurious_num, generate_num = generate_num, data = d_))
    return prompts

def get_batch_response(model:str, prompts:list, batch = 30,temperature=0, max_token=8192):
    if "llama" in model:
        # meta.llama3-1-8b-instruct-v1:0
        response = []
        for i in tqdm.tqdm(range(math.ceil(len(prompts)/batch))):
            # print(i)
            temp_prompt = prompts[i*batch:(i+1)*batch]
            try: 
                r = get_model_batch_response(prompts=temp_prompt,model=model,temperature=temperature,max_new_tokens=max_token)
                response = response + r
                break
            except Exception as e:
                print(e)
        
        return response

def get_raw_response():
    prompt = prompt_t
    model = "meta.llama3-1-70b-instruct-v1:0"
    task = 'task1'
    description = "description1"
    generate_num = 10
    spurious_num = 3
    with open('../results/baseline/checkpoint-216/log_dev_wrong_test_data.json',"r") as f:
        data = json.load(f)
    prompts = format_prompt(data,prompt,task,description,spurious_num,generate_num)
    print(prompts[0])
    response = get_batch_response(model,prompts,batch = 1)
    return 
    



KeyboardInterrupt: 

In [5]:
import json
import argparse
import re
from sympy import EX
from prompt import get_prompt, get_task_name, get_task_description
from response import get_batch_response
import csv
import os
import pandas as pd
from find_path import find_path
from utils import get_model_batch_response
import math
import tqdm 

prompt_t = """ 
You need to augment the data to four new data for training a student in stance detection. Stance detection aims to identify the authors' attitudes or positions [FAVOR, NONE, AGAINST] towards a specific target such as an entity, a topic.

You are given the following JSON data:
{data}

Your task is to generate {k} new XML examples that follow the same structure as the provided JSON. Each new example must have the same elements as the original JSON data.

The augmented data are for the directions:
1. Minimally alter certain words to shift the stance of the given data to two other labels, resulting in two new samples.
2. Modify the wording of the original sentence with a similar sentence length and a similar meaning, without changing the target and the label.
3. Shift the focus to a different target while assigning the corresponding stance, ensuring that the sentence structure remains consistent with the original.

For each new example:
1. The structure of the XML should match the original JSON, with the same elements.
2. The format and types (e.g., numbers, strings, lists) should remain consistent with the original data, and the lengths should be similar. Donot make the expression of the stance too straightforward, with an explicit expression of love, like, neutral, or dislike and so on.
3. The new XML data should be valid and realistic for the context of the task.
4. The ground_truth should be one of [FAVOR,NONE,AGAINST].

Please output the generated data in the following XML format, where each element corresponds to a key in the original JSON:

```xml
<data>
    <example_1>
        <text>Regulation of corporations has been subverted by corporations. States that incorporate corporations are not equipped to regulate corporations that are rich enough to influence elections, are rich enough to muster a legal team that can bankrupt the state. Money from corporations and their principals cannot be permitted in the political process if democracy is to survive.</text>
        <target>company</target>
        <ground_truth>AGAINST</ground_truth>
    </example_1>
    <example_2>
        <text>The whole media mess surrounding the royals is a consequence of the promotional fervor with which royal households (aka, public relations experts) developed stage-set performances for the public to devour. Prior to the Victorian era, those elaborate and lethally expensive weddings, coronations, and funerals - and the fairy tales that went along with them - just didn't exist.</text>
        <target>flag burning</target>
        <ground_truth>NONE</ground_truth>
    </example_2>
    <example_3>
        <text>Two of the main reasons people switch to cannabis from pharmaceuticals and other drugs such as alcohol: less side-effects and less withdrawal: ""Over 41% state that they use cannabis as a substitute for alcohol, 36.1% use cannabis as a substitute for illicit substances, and 67.8% use cannabis as a substitute for prescription drugs. The three main reasons cited for cannabis-related substitution are 'less withdrawal' (67.7%), 'fewer side-effects' (60.4%), and 'better symptom management' suggesting that many patients may have already identified cannabis as an effective and potentially safer adjunct or alternative to their prescription drug regimen."" [Lucas et al. Cannabis as a substitute for alcohol and other drugs: A dispensary-based survey of substitution effect in Canadian medical cannabis patients. Addiction Research & Theory. 2013]</text>
        <target>marijuana</target>
        <ground_truth>FAVOR</ground_truth>
    </example_3>
    ...
</data>
"""


def format_prompt(data,prompt, task,description,spurious_num,generate_num):
    # NOTE
    # model's label { "FAVOR": 0  , "NONE": 1 , "AGAINST":  2}
    # the data is from model's inference
    # but that is needed to transform to { "FAVOR": 1, "NONE": 2, "AGAINST": 0}
    transform_dict = {
        0: 'FAVOR',
        1: 'NONE',
        2: 'AGAINST'
    }
    prompts = []
    for d in data:
        d_ = {
            "input_text": d["input_text"],
            "target": d["target"],
            "true_label": transform_dict[d["true_label"]]
        }
        prompts.append(prompt_t.format(data = d_,k = 4))
    return prompts

def get_batch_response(model:str, prompts:list, batch = 30,temperature=0, max_token=8192):
    if "llama" in model:
        # meta.llama3-1-8b-instruct-v1:0
        response = []
        for i in tqdm.tqdm(range(math.ceil(len(prompts)/batch))):
            # print(i)
            temp_prompt = prompts[i*batch:(i+1)*batch]
            try: 
                r = get_model_batch_response(prompts=temp_prompt,model=model,temperature=temperature,max_new_tokens=max_token)
                response = response + r
                break
            except Exception as e:
                print(e)
        
        return response

def get_raw_response():
    prompt = prompt_t
    model = "meta.llama3-1-70b-instruct-v1:0"
    task = 'task1'
    description = "description1"
    generate_num = 4
    spurious_num = 3
    with open('../results/baseline/checkpoint-216/log_dev_wrong_test_data.json',"r") as f:
        data = json.load(f)
    prompts = format_prompt(data,prompt,task,description,spurious_num,generate_num)
    print(prompts[0])
    response = get_batch_response(model,prompts,batch = 1)
    return response

r = get_raw_response()

print(r[0])

 
You need to augment the data to four new data for training a student in stance detection. Stance detection aims to identify the authors' attitudes or positions [FAVOR, NONE, AGAINST] towards a specific target such as an entity, a topic.

You are given the following JSON data:
{'input_text': "Look, there are issues to be debateed. Do we need as much brick and mortaror public shelf space as we have? I use libraries a lot. I download audible books frequently, but I don't use my local northern NY library system to do it, I use the NY City Library. If I do want a book I locate it on Amazon (since its presentation of the book: description, professional and reader reviews is much better than the libraries) then I order it (on-line) from my local library system. When I get the e-mail notice that the book is ready I finally pay a visit to my library to pick the book up. Using this process I wonder why my library has publically displayed bookshelves at all.", 'target': 'library', 'true_label':

  0%|          | 0/529 [00:14<?, ?it/s]



Here are the four new XML examples that follow the same structure as the provided JSON:

```xml
<data>
    <example_1>
        <text>I'm a big fan of libraries. I use them frequently to borrow books, attend events, and study. I love how they provide a quiet and comfortable space for people to learn and grow. I also appreciate how they offer free access to resources and information that might be otherwise unavailable to me.</text>
        <target>library</target>
        <ground_truth>FAVOR</ground_truth>
    </example_1>
    <example_2>
        <text>I'm not sure what the point of libraries is anymore. With the rise of e-books and online resources, I don't see the need for physical bookshelves. I mean, I can just download a book on my phone or laptop and read it wherever I want. I don't need to go to a library to do that.</text>
        <target>library</target>
        <ground_truth>NONE</ground_truth>
    </example_2>
    <example_3>
        <text>I recently visited my local library